In [ ]:
#####
# Initial cell
#####
import ipywidgets as widgets
from IPython.display import display
from collections.abc import Callable


def multiple_choice(question: str, options: list, correct_option) -> widgets.Widget:
    options_widget = widgets.ToggleButtons(
        options=options,
        value=None,
        #    layout={'width': 'max-content'}, # If the items' names are long
        disabled=False
    )

    def eval_func(widget):
        if widget.value is None:
            return None
        return widget.value == correct_option

    return generic_question(question, options_widget, eval_func)


def multiple_answers(question: str, options: list, correct_answers: list) -> widgets.Widget:
    buttons = [widgets.ToggleButton(
        value=False, description=option) for option in options]

    def feedback(evaluation_result):
        if evaluation_result == None:
            return "Please pick an answer"
        elif evaluation_result == 0:
            return "wrong"
        else:
            return f"Correct answers: {evaluation_result}/{len(correct_answers)}"

    def eval_func(widget: widgets.HBox):
        answers = set(
            button.description for button in widget.children if button.value)
        if len(answers) == 0:
            return None
        # Evaluates number of correct choices minus number of incorrect choices.
        return len(answers.intersection(correct_answers)) - len(answers.difference(correct_answers))

    return generic_question(question, widgets.HBox(buttons), eval_func, feedback)


def standard_feedback(evaluation_result: int):
    if evaluation_result == None:
        return "No answer selected"
    elif evaluation_result == 0:
        return "Wrong answer"
    else:
        return "Correct"


def generic_question(question, input_widget, 
                     evaluation_function: Callable[[widgets.Widget], bool | None], 
                     feedback=standard_feedback) -> widgets.Widget:

    title_widget = widgets.HTMLMath(value=f"<h3>{question}</h3>")
    output = widgets.Output()

    def _inner_check(button):
        with output:
            output.outputs = [
                {'name': 'stdout', 'text': feedback(evaluation_function(input_widget)), 'output_type': 'stream'}]

    button = widgets.Button(description="Check answer", icon="check",
                            style=dict(
                                button_color="lightgreen"
                            ))
    button.on_click(_inner_check)
    layout = widgets.VBox([title_widget,
                           widgets.HBox([input_widget],
                                        layout=widgets.Layout(padding="10px 20px 10px 20px", border="solid")),
                           widgets.VBox([button, output],
                                        layout=widgets.Layout(margin="10px 10px 0px 0px"))])

    return layout


def numeric_input(question: str, correct_answer: float) -> widgets.Widget:

    input_widget = widgets.FloatText(
        value=None,
    )
    def eval_func(widget): 
        if widget.value is None:
            return None
        return widget.value == correct_answer
    return generic_question(question, input_widget, eval_func)


def code_question(question: str, expected_outputs) -> widgets.Widget:
    """
    params:
    - expected_output - a list of pairs in the format:
        - ((inputs), (expected_output))
        - Example: [
            ((2, 4), 8)
        ]
    """

    input_widget = widgets.Text(
        description="What is the name of your function?", placeholder="myFunction",
        style=dict(description_width="initial"))

    def eval_func(widget):
        function_name = widget.value
        if function_name not in globals():
            # Error handling
            return None

        function = globals()[function_name]
        return all([function(*test_input) == test_output
                    for test_input, test_output in expected_outputs])
    
    def feedback(evaluation_result):
        if evaluation_result is None:
            return "No function defined with that name. Remember to run the cell to define the function."
        if evaluation_result:
            return "Correct"
        else:
            return "Incorrect answer"

    return generic_question(question, input_widget, eval_func, feedback)


def display_json() -> widgets.Widget:
    # Delegating

    pass

# Test quiz

In [56]:
# Define your function for task 1 here:

def test(x, y):
    return 2+x+y

def tet(x, y):
    return x+y

In [67]:
display(widgets.HTML("<h2>Test your knowledge</h2>"))

display(multiple_choice("Hva heter du?", ["Jakob", "Jørgen", "Ravi"], "Jakob"))

display(numeric_input("What is 2 + 5?", 7))

display(numeric_input("What is 2/4 + 1/4?", 0.75))

display(code_question(
    "Make a function that returns the sum of two numbers", [((2, 2), 4)]))

display(multiple_answers("Which of these are cheeses?",
                         options=["Gouda", "Emmentaler", "Brie", "Cat", "Dog"],
                         correct_answers=["Gouda", "Emmentaler", "Brie"]))

# buttons = [widgets.ToggleButton(value=False, description=option) for option in ["Gouda", "Emmentaler", "Brie", "Cat", "Dog"]]
# display(buttons)
# import time
# time.sleep(20)
# print("Choose now")
# time.sleep(10)

# print([button.description for button in buttons if button.value])


HTML(value='<h2>Test your knowledge</h2>')

In [58]:
# # title = widgets.Text(value="What is the best pizza topping")
# title = widgets.HTML(value="<h2>What is the best topping?</h2>")
# buttons = widgets.RadioButtons(
#     options=['pepperoni', 'pineapple', 'anchovies'],
#    value=None, # Defaults to 'pineapple'
# #    layout={'width': 'max-content'}, # If the items' names are long
#     disabled=False
# )

# from urllib.request import urlopen

# # file = urlopen("https://plus.unsplash.com/premium_photo-1664474619075-644dd191935f?fm=jpg&q=60&w=3000&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxzZWFyY2h8MXx8aW1hZ2V8ZW58MHx8MHx8fDA%3D")
# file = open("../figures/image.jpg", "rb")
# image = file.read()
# image = widgets.Image(
#     value=image,
#     format='jpg',
#     width=300,
#     height=400,
# )

# def f(value):
#     if value == None:
#         print("Answer!")
#     # print(buttons)
#     # value = buttons.value
#     elif value == "pepperoni":
#         print("Correct!")
#     else:
#         print("Incorrect!")

# display(title)
# display(image)
# display(buttons)
# display(widgets.interactive_output(f, {"value": buttons}))

# output = widgets.Output()
# button = widgets.Button(description="Check answer")
# def _inner_check(button):
#     with output:
#         if "test" not in globals():
#             output.outputs = [{'name': 'stdout', 'text': 'Please define the function in the cell above!', 'output_type': 'stream'}]
#         elif test(2) == 4:
#             output.outputs = [{'name': 'stdout', 'text': 'Correct!', 'output_type': 'stream'}]
#         else:
#             output.outputs = [{'name': 'stdout', 'text': 'Incorrect!', 'output_type': 'stream'}]
# button.on_click(_inner_check)
# display(button, output)
